# **Learned features в CNN: Практика**

Добро пожаловать в практическую часть по первой части модуля ["Методы объяснения в DL"](https://stepik.org/a/198640)!

Повторим пройденные тезисы:

1. Сверточные нейронные сети извлекают паттерны изображения благодаря skip-connections и сверточным слоям
2. Извлеченные из изображения структуры для обученной сверточной сети можно увидеть,пропустив пример через слои свертки с весами последовательно
3. Сверточные нейронные сети способны извлекать понятные человеку концепции из данных

Чтобы убедится в каждом, перейдем к практике!

**Quiz 1. Какой из тезисов 1-3, предложенных в начале, содержит ошибку?**


In [ ]:
!pip install torch torchvision -q


In [ ]:
import torch
import requests
import numpy as np
from tqdm import tqdm
from io import BytesIO
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt

Будем работать с ResNet18, также известной как Residual Network — достаточно популярной архитектурой для задачи классификации. Основная особенность ResNet — использование остаточных связей (skip connections), что позволяет эффективно обучать очень глубокие сети, избегая проблемы затухания градиентов.

В ResNet18, согласно названию, используется 18 слоёв. Размер входного слоя сети 224 x 224. После, грубо говоря, принятия изображения в сеть последовательно применяются слои свертки c MaxPooling, Batch Normalization и функциями активации.




## Архитектура ResNet18

![temp-Imageb9-Tg-EC.avif](https://www.researchgate.net/profile/Sajid-Iqbal-13/publication/336642248/figure/fig1/AS:839151377203201@1577080687133/Original-ResNet-18-Architecture.png)

Загрузим преобученную модель из pyTorch и посмотрим на её архитектуру.

In [ ]:
# Загрузка модели
model = models.resnet18(pretrained=True)

#Рассмотрение архитектуры
model

**Quiz 2: Какая функция активации применяется в ResNet18?** \
**Quiz 3: Сколько слоев свертки содержит данная архитектура ResNet18?** \

Вспомним ешё несколько деталей из модуля:

- Первые (от входного слоя) сверточные слои изучают абстрактные составляющие
- Следующие слои изучают отдельные текстуры
- Ближе к последним сверточным слоям в извлекаемой информации просматриваются паттерны,
- В конце можно увидеть изображения

также, мы зафиксировали, что эти концепции возможно увидеть в любой обученной сверточной нейроной сети, применяя слои свертки с их весами к данным последовательно.

Тогда сформулируем, что нам нужно:

1. Рассмотреть обученную сеть $net(img)$, содержающую $n$ слоев свертки с фиксированными весами $w_i, i=\vec{1, n}$.
2. Для каждого слоя внутри сети
    - если он является сверточным, извлечь слой с его параметрами
    - иначе пойти дальше

Для решения задачи мы могли бы написать пройстой цикл, но архитектура ResNet18 предполагает, что свертки являются частями последовательного (*Sequential*) слоя. Учитывая это, наш алгоритм придется немного преобразовать:

1. Рассмотреть обученную сеть $net(img)$, содержающую $n$ слоев свертки с фиксированными весами $w_i, i=\vec{1, n}$.
2. Для каждого слоя внутри сети
    - если он является сверточным, извлечь слой с его параметрами
    - иначе если слой является Sequential
        - рассмотреть каждую его составляющую
        - если составляющая является сверточным слоем, извлечь её с её параметрами
        - иначе пойти дальше

И таким способом нам надо пройтись по сети. Сделаем ниже в коде.

В действительности, для визуализации мы не будем извлекать все слои, а ограничимся меньшим их числом — 17ю. Делать мы это будем потому что 3 слоя свертки содержатся в подблоке *downsamle*.

![temp-Imagermy-OBM.avif](https://i.postimg.cc/25Bg8bVX/temp-Imagermy-OBM.avif)

 Оператор `print` этого не отражает, но свертки в downsmapling применяются не линейно, а парралельно сверткам выше в базовом блоке (`BasicBlock`). Их параметры пробрасываются в следующий блок, поэтому не извлекая их отдельно мы ничего не теряем.

In [ ]:
# Списки для извлечения сверточных слоев и их весов
conv_weights = [] # список для весов слоя
conv_layers = [] # список для хранения самих слоев

counter = 0 # счетчик для того, чтобы убедиться, что мы извлекли все слои

# сохраним все компоненты модели в список
model_children = list(model.children())


for i in range(len(model_children)):
    if type(model_children[i]) == nn.Conv2d:
        counter+=1
        conv_weights.append(model_children[i].weight)
        conv_layers.append(model_children[i])
    elif type(model_children[i]) == nn.Sequential:
      for j in range(0, len(model_children[i])):
        for child in model_children[i][j].children():
          if type(child) == nn.Conv2d:
                    counter+=1
                    conv_weights.append(child.weight)
                    conv_layers.append(child)
          elif type(child) == nn.Sequential and len(list(child.children())) != 0:
            for k in range(0, len(list(child.children()))):
              if type(list(child.children())[k]) == nn.Conv2d:
                counter+=1
                conv_weights.append(list(child.children())[k].weight)
                conv_layers.append(list(child.children())[k])


print(f'Извлечено слоёв: {counter}')

Извлечено слоёв: 20


**Quiz 4:** Исправьте код ниже так, чтобы он извлек все сверточные слои, кроме тех, что содержатся в downsmaple подблоках. Сколько слоев у вас получилось после исправления? \

In [ ]:
# Списки для извлечения сверточных слоев и их весов
conv_weights = [] # список для весов слоя
conv_layers = [] # список для хранения самих слоев

counter = 0 # счетчик для того, чтобы убедиться, что мы извлекли все слои

# сохраним все компоненты модели в список
model_children = list(model.children())

for i in range(len(model_children)):
    if type(model_children[i]) == nn.Conv2d:
        counter+=1
        conv_weights.append(model_children[i].weight)
        conv_layers.append(model_children[i])
    elif type(model_children[i]) == nn.Sequential:
      for j in range(0, len(model_children[i])):
        for child in model_children[i][j].children():
          if type(child) == nn.Conv2d:
                    counter+=1
                    conv_weights.append(child.weight)
                    conv_layers.append(child)
          elif type(child) == nn.Sequential and len(list(child.children())) != 0:
            for k in range(0, len(list(child.children()))):
              if type(list(child.children())[k]) == nn.Conv2d:
                counter+=1
                conv_weights.append(list(child.children())[k].weight)
                conv_layers.append(list(child.children())[k])


print(f'Извлечено слоёв: {counter}')


**Входное изображение.**

Загрузим конкретный пример $x_0$.

In [ ]:
# Загрузка изображения
url = 'https://github.com/aiedu-courses/all_datasets/blob/main/images/pig.png?raw=true'

image_bytes = requests.get(url).content
image = Image.open(BytesIO(image_bytes))
image = image.convert("RGB")
plt.imshow(image)
plt.show()

In [ ]:
print(f"Original image size: {image.size}")

Original image size: (2066, 1438)


In [ ]:
np.array(image).shape

(1438, 2066, 4)

По умолчанию png имеет 4 канала, 4й — отвлечает за прозрачность пикселя. Исправьте изображение так, чтобы работать с RGB каналами.

In [ ]:
#Ваш код здесь

На вход ResNet18 может быть подано изображение любого разрешения. Проходя свертки последовательно и благодаря `AdaptiveAvgPool2d` изображение обработается в любом случае. Однако, чтобы облегчить и ускорить обработку изображения, мы изменим его размеры до 1024x1024 и нормализуем.

Обратите внимание, что mean и std мы задаем сразу, не вычисляя. Для моделей, обученных на Imagenet данные значения являются обычной практикой. Они были рассчитаны на основе изображений датасета.

In [ ]:
#Средние
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize((1224, 1224)),  # Изменяем размер изображения
    transforms.ToTensor(),  # Конвертируем картинку в pyTorch тензор
    transforms.Normalize(mean=mean, std=std)  # Нормализуем картинку
])

image = transform(image)
print(f"Image shape after resizing: {image.shape}")

Image shape after resizing: torch.Size([3, 1224, 1224])


In [ ]:
# Переместим модель на GPU iесли это возможно
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)
image = image.to(device)

# Определим списки для извлечения feature maps
feature_maps = []  # Список для feature maps
layer_names = []  # Список для layer names


for number, layer in tqdm(enumerate(conv_layers)):
    proccess = round((number+1)/17*100, 2)
    image = layer(image)
    feature_maps.append(image)
    layer_names.append(str(number+1)+'_'+str(layer))



17it [00:08,  2.07it/s]


In [ ]:
# Предобработка feature maps к визуализируемому виду
processed_feature_maps = [] # Список для хранения полученных карт активации
for feature_map in feature_maps:
	feature_map = feature_map.squeeze(0) # Удаляем размерность батча
	mean_feature_map = torch.sum(feature_map, 0) / feature_map.shape[0] # Вычисляем среднее по каналам
	processed_feature_maps.append(mean_feature_map.data.cpu().numpy())

Наконец, отобразим получившиеся результаты.

In [ ]:
# Отобразим карты признаков
fig = plt.figure(figsize=(30, 30))
for i in range(len(processed_feature_maps)):
	ax = fig.add_subplot(5, 4, i + 1)
	ax.imshow(processed_feature_maps[i])
	ax.axis("off")
	ax.set_title(layer_names[i].split('(')[0], fontsize=25)


**Quiz 5: Почему хрюша получилась зелёной?**

**Quiz 6: Поэкспериментируйте с `transfroms.Resize` в `transform`. За что он отвечает в данном случае?**